<a href="https://colab.research.google.com/github/albertomanfreda/intensive_school_ml/blob/master/lessonScipy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SciPy

"SciPy is a collection of mathematical algorithms and convenience functions built on the NumPy extension of Python" (from SciPy documentation).

SciPy is organized in several subpackages. Here we will take a quick look at a few of them. You can find the full list at https://docs.scipy.org/doc/scipy/reference/tutorial/general.html

The modules we will look at are

 * stats
 * optimize
 * interpolation
 *integrate  

## The stats subpackage

**scipy.stats** contains a collections of statistical distributions, both discrete and continue, both univariate and multivariate, as well as a respectable number of statistical functions.

These are not just plain functions, but rather objects with several functionalities: *probability density function* (pdf), *cumulative density function* (cdf), *percent probability function* (ppf), *moments*, *median* etc...

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import scipy.stats
from matplotlib import pyplot as plt

x_grid = np.linspace(-6., 8., 100)

# Instantiate a gaussian function with mean 1. and std. dev. 2
gauss_dist = scipy.stats.norm(loc=1.0, scale=2.)

# Calculate some statistical variables with the stats() method
mean, var, skewness = gauss_dist.stats(moments='mvs')
print('mean = {:.3f}, variance = {:.3f}, skewness = {:.3f}'.\
      format(mean, var, skewness))

# Draw the pdf, using the pdf() method
plt.figure('pdf')
plt.plot(x_grid, gauss_dist.pdf(x_grid), label='gaussian pdf')
plt.legend()

# Draw the cdf
plt.figure('cdf')
plt.plot(x_grid, gauss_dist.cdf(x_grid), label='gaussian cdf')
plt.legend()

plt.show()



Bottom line: before wrting a function yourself, take the time to check if it is already in SciPy - as it probably will.

## Fitting tools

SciPy provides three different ways of fitting: the **optimize** subpackage contains the *curve_fit* function for least square algorithm, while **odr** contains the ODR routine for orthogonal distance regression. Finally, the distribution of the **stats** module are able to *fit* themselves to data using Maximum Likelihood Estimate.

Note: **optimize** also contains many kind of root-finding and minimization rountines, which are useful in a variety of situations.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit


# Define the fit fnction: a simple line
def line(x, m, q):
    return m*x + q

# This time we will load data from a file:
x, y, dx, dy = np.loadtxt('sample_fit_data.txt')

# Draw the data
plt.errorbar(x, y, yerr=dy, xerr=dx, fmt='ko', label='data')

# Setup the axes
plt.xlabel('x')
plt.ylabel('y')

""" curve_fit assumes that the first positional parameter of the function 
is the independent variable, while the others are the free parameters.
It returns two items: the first is a tuple with the optimal prameters
found, while the second is the covriance matrix, which is a n x n matrix,
where n is the number of free parameters.
""" 
popt, pcov = curve_fit(line, x, y, sigma=dy)
# The errors can be estimated by the square root of the paramater variances,
# which are on te diagonal of the covariance matrix
p_errs = np.sqrt(np.diagonal(pcov))
for p, p_err in zip(popt, p_errs):
    print('{:.4f} +- {:.4f}'.format(p, p_err))

# Draw the model (we need a grid of dat point)
x_grid = np.linspace(min(x), max(x), 10)
plt.plot(x_grid, line(x_grid, *popt), 'r-', label='curve_fit')

plt.legend()
plt.show()


## Interpolation

Interpolation is a surprisingly common task. In the **scipy.interpolate** subpackage, SciPy provides a powerful tool for interpolation - **splines** - both unidimensional or with higher dimensionality.

A spline is an object that is built starting from a number of input points, and interpolates through them by defining a picewise polinomial function of a given degree. Commonly used degrees are 1 (linear interpolation) or 3 (cubic interpolation).

In [ ]:
import numpy as np
import scipy.interpolate
from matplotlib import pyplot as plt

# Generate some data
x = np.arange(10)
y = np.sin(x)
# Draw the data
plt.plot(x, y, 'ko', label='data')

# Setup the axes
plt.xlabel('x')
plt.ylabel('sin(x)')

# Create a linear splie to interpolate the data
linear_spline = scipy.interpolate.InterpolatedUnivariateSpline(x, y, k=1)
# Create a cubic spline to interpolate the data
cubic_spline = scipy.interpolate.InterpolatedUnivariateSpline(x, y, k=3)

# A spline can be called like a normal function
print('Cubic spline value at x=4.5 is {:.5f}'.format(cubic_spline(4.5)))

# Create a grid of points where the splines will be evaluated
x_grid = np.linspace(min(x), max(x), 30)

# Draw the 2 splines
plt.plot(x_grid, linear_spline(x_grid), 'r-', label='linear')
plt.plot(x_grid, cubic_spline(x_grid), 'b--', label='cubic')

# Draw the grid an the legend, then plot everything
plt.legend()
plt.grid(True)
plt.show()



## The integrate subpackage

**scipy.integrate** contains method for numerical integration of functions and numerical solution of ordinary differential equations (ODE), such as Runge-Kutta methods. 



In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import integrate

# define the function we want to integrate
# This can be a spline as well
def integrand(x):
    return np.exp(-0.4 * x) * np.cos(2 * x)

# Draw it
x = np.linspace(-5., 6., 200)
plt.figure(figsize=(8, 6))
plt.plot(x, integrand(x))
plt.grid(True)

# Define the integration extrema
integr_extrema = (-4.5, -1.)

# Draw vertical lines in their corrispondence in the plot
plt.axvline(integr_extrema[0], color='gray', linestyle='--')
plt.axvline(integr_extrema[1], color='gray', linestyle='--')
#Fill the integral area
plt.fill_between(x, integrand(x), y2=0,
                 where=(x > integr_extrema[0]) * (x < integr_extrema[1]))

# Calculate the actual integral
value, error = integrate.quad(integrand, integr_extrema[0], integr_extrema[1])
# Write the result on screen.
plt.text(0, -4, '$\int_{{{:.1f}}}^{{{:.1f}}} f(x) = {:.6f} \pm {:.2g}$'.format(
         integr_extrema[0], integr_extrema[1], value, error), fontsize=14)

# Draw everything
plt.show()